In [ ]:
#Deep learning model to predict stock values based on historic data
#We have used LSTM to predict next 30 days of stock values, based on 15 day lag values.
#reference : https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/
#Reference : https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816


In [1]:
#importing libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import layers
%matplotlib inline
from datetime import datetime as dt
#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense

Using TensorFlow backend.
/Users/apoorvabanubakode/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/apoorvabanubakode/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/apoorvabanubakode/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/apoorvabanuba

In [2]:
#read data for all the avaolable stocks, from the below csv 

In [3]:
dffull=pd.read_csv('/Users/apoorvabanubakode/Downloads/stockdata-4.csv',)
#df.columns=['Name','Date','Open','High','Volume','Change','ChangePercent']
dffull.head()
#We will be using only the close value in prediction instead of all these and 5 years of data.

,company,date,open,close,volume,change,changePercent
0,DIS,2015-05-11,113.94,112.50,6865026,0.00,0.0000
1,DIS,2015-05-12,112.26,109.44,6494548,0.66,0.6080
2,DIS,2015-05-13,111.90,110.59,4177183,-0.05,-0.0464
3,DIS,2015-05-14,112.99,111.58,4765440,0.75,0.6836
4,DIS,2015-05-15,114.70,112.30,6106343,0.37,0.3483


In [23]:
def predict_next30days(nextdays,laststockdate):
        predictionli=[]
        predictionli = df[-15:]
        for i in range(30):
            x = predictionli[-15:]
            x = x.reshape((1, 15, 1))
            predictedval = model.predict(x)[0][0]
            predictionli = np.append(predictionli, predictedval)
        predictionli = predictionli[14:]
        dates = pd.date_range(laststockdate, periods=nextdays+1).tolist()
    
        return predictionli, dates

In [25]:
un=dffull.company.unique()
un

array(['DIS', 'NVO', 'MSFT', 'GOOGL', 'NEE', 'BEP', 'ENPH', 'AMZN',
       'VEEV', 'TER', 'NVDA', 'NFLX', 'VRTX', 'NOW', 'ADBE', 'VTI',
       'IXUS', 'ILTB', 'ZEN', 'T', 'VZ', 'AAXN', 'TERP', 'IRBT', 'W',
       'STZ', 'CTRE', 'CVS', 'FDX', 'ALL', 'AZO', 'ALB', 'BTI', 'VIAC',
       'ALXN', 'URI', 'ETN'], dtype=object)

In [27]:
alllist=[]
#allist will have stocks of all the companies 
for company in un:
    #for every company do the below 
    df=dffull[dffull.company==company]
    #change into datetime stamp - column
    df['date'] = pd.to_datetime(df['date'])
    #backup of df 
    backupdf=df.copy()
    #fetch the last date for which stock is available
    laststockdate=backupdf.iloc[-1,1]
    df.set_axis(df['date'], inplace=True)
    #drop columns 
    df.drop(columns=['open', 'change', 'volume','changePercent','company'], inplace=True)
    print(df.head())
    df = df['close'].values
    df = df.reshape((-1,1))
    print(df)
    traindf = df[:int(df.shape[0]*0.80)]
    testdf = df[int(df.shape[0]*0.80):]
    train_generator_ts = TimeseriesGenerator(traindf, traindf, length=15, batch_size=20)     
    test_generator_ts = TimeseriesGenerator(testdf, testdf, length=15, batch_size=1)
    
    model = Sequential()
    
    model.add(
    LSTM(100,
        activation='relu',
        input_shape=(15,1))
    )
    
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit_generator(train_generator_ts, epochs=10, verbose=1)
    res = model.predict_generator(test_generator_ts)
    
    traindf = traindf.reshape((-1))
    testdf = testdf.reshape((-1))
    res = res.reshape((-1))
    df = df.reshape((-1))
    
    forecastedres30days,dates = predict_next30days(30,laststockdate)

    alllist.append(forecastedres30days)
    print(forecastedres30days)

/Users/apoorvabanubakode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


                 date   close
date                         
2015-05-11 2015-05-11  112.50
2015-05-12 2015-05-12  109.44
2015-05-13 2015-05-13  110.59
2015-05-14 2015-05-14  111.58
2015-05-15 2015-05-15  112.30
[[112.5 ]
 [109.44]
 [110.59]
 ...
 [109.26]
 [108.  ]
 [108.  ]]
Epoch 1/10
72/72 [==============================] - 1s 14ms/step - loss: 1282.8465
Epoch 2/10
72/72 [==============================] - 1s 7ms/step - loss: 351.1107
Epoch 3/10
72/72 [==============================] - 1s 7ms/step - loss: 254.1228
Epoch 4/10
72/72 [==============================] - 1s 7ms/step - loss: 249.7601
Epoch 5/10
72/72 [==============================] - 1s 7ms/step - loss: 213.0160
Epoch 6/10
72/72 [==============================] - 1s 7ms/step - loss: 199.1229
Epoch 7/10
72/72 [==============================] - 1s 8ms/step - loss: 210.1691
Epoch 8/10
72/72 [==============================] - 1s 7ms/step - loss: 185.8243
Epoch 9/10
72/72 [==============================] - 1s 8ms/step - loss: 19

                 date  close
date                        
2015-05-11 2015-05-11  85.08
2015-05-12 2015-05-12  84.26
2015-05-13 2015-05-13  83.53
2015-05-14 2015-05-14  84.58
2015-05-15 2015-05-15  91.09
[[ 85.08]
 [ 84.26]
 [ 83.53]
 ...
 [440.3 ]
 [427.12]
 [427.12]]
Epoch 1/10
72/72 [==============================] - 1s 19ms/step - loss: 14226.1965
Epoch 2/10
72/72 [==============================] - 1s 7ms/step - loss: 1624.8314
Epoch 3/10
72/72 [==============================] - 1s 7ms/step - loss: 1311.9505
Epoch 4/10
72/72 [==============================] - 1s 8ms/step - loss: 1276.4000
Epoch 5/10
72/72 [==============================] - 1s 7ms/step - loss: 1074.4208
Epoch 6/10
72/72 [==============================] - 1s 7ms/step - loss: 1023.5168
Epoch 7/10
72/72 [==============================] - 1s 7ms/step - loss: 1157.1413
Epoch 8/10
72/72 [==============================] - 1s 7ms/step - loss: 1309.1333
Epoch 9/10
72/72 [==============================] - 1s 8ms/step - loss: 9

Epoch 1/10
72/72 [==============================] - 2s 24ms/step - loss: 42.1449
Epoch 2/10
72/72 [==============================] - 1s 8ms/step - loss: 26.4992
Epoch 3/10
72/72 [==============================] - 1s 7ms/step - loss: 10.7730
Epoch 4/10
72/72 [==============================] - 1s 8ms/step - loss: 9.3217
Epoch 5/10
72/72 [==============================] - 1s 8ms/step - loss: 10.6992
Epoch 6/10
72/72 [==============================] - 1s 8ms/step - loss: 11.0501
Epoch 7/10
72/72 [==============================] - 1s 8ms/step - loss: 8.8256
Epoch 8/10
72/72 [==============================] - 1s 8ms/step - loss: 6.1245
Epoch 9/10
72/72 [==============================] - 1s 8ms/step - loss: 7.8721
Epoch 10/10
72/72 [==============================] - 1s 8ms/step - loss: 5.3599
[17.31       16.47345161 16.34487152 16.29840851 16.22721863 16.15827751
 16.07216263 15.96695614 15.86365509 15.75719929 15.64513588 15.54129028
 15.42852592 15.31163597 15.21227455 15.11511326 15.01756

                 date   close
date                         
2015-05-11 2015-05-11  175.30
2015-05-12 2015-05-12  181.44
2015-05-13 2015-05-13  175.30
2015-05-14 2015-05-14  175.63
2015-05-15 2015-05-15  178.70
[[175.3 ]
 [181.44]
 [175.3 ]
 ...
 [129.88]
 [121.5 ]
 [121.5 ]]
Epoch 1/10
72/72 [==============================] - 2s 27ms/step - loss: 3797.3161
Epoch 2/10
72/72 [==============================] - 1s 8ms/step - loss: 1331.3983
Epoch 3/10
72/72 [==============================] - 1s 8ms/step - loss: 1087.0127
Epoch 4/10
72/72 [==============================] - 1s 8ms/step - loss: 802.0361
Epoch 5/10
72/72 [==============================] - 1s 8ms/step - loss: 767.8340
Epoch 6/10
72/72 [==============================] - 1s 8ms/step - loss: 723.7617
Epoch 7/10
72/72 [==============================] - 1s 8ms/step - loss: 783.8517
Epoch 8/10
72/72 [==============================] - 1s 8ms/step - loss: 681.5826
Epoch 9/10
72/72 [==============================] - 1s 8ms/step - loss: 

72/72 [==============================] - 1s 8ms/step - loss: 55.1585
Epoch 8/10
72/72 [==============================] - 1s 8ms/step - loss: 48.9831
Epoch 9/10
72/72 [==============================] - 1s 8ms/step - loss: 47.5211
Epoch 10/10
72/72 [==============================] - 1s 8ms/step - loss: 40.8209
[17.15       14.12036324 14.30844593 14.33109379 14.27599049 14.1620779
 13.97775269 13.6965313  13.37601757 13.03822517 12.68325806 12.25560284
 11.8547411  11.34311676 10.91151142 10.49037933 10.04185963  9.70558929
  9.32489014  8.9145546   8.47952175  8.02543545  7.55765104  7.08344507
  6.60580921  6.12766695  5.65112638  5.18006134  4.71541071  4.2643714
  3.82712078]
                 date   close
date                         
2015-05-11 2015-05-11  161.56
2015-05-12 2015-05-12  161.25
2015-05-13 2015-05-13  163.72
2015-05-14 2015-05-14  166.00
2015-05-15 2015-05-15  167.59
[[161.56]
 [161.25]
 [163.72]
 ...
 [111.31]
 [105.68]
 [105.68]]
Epoch 1/10
72/72 [===================

In [28]:
for val in alllist:
    print((val))

[108.         101.50862122  99.75052643  99.58942413  99.16755676
  98.98065948  99.14421082  98.73390961  97.87608337  97.31177521
  96.48106384  95.41429901  94.71022034  93.95301819  93.21143341
  92.4081955   91.45420074  90.72413635  90.14556885  89.56625366
  88.98535156  88.42817688  87.81798553  87.16704559  86.51019287
  85.83849335  85.20114136  84.61383057  84.03240967  83.44828033
  82.86601257]
[66.08       67.03127289 67.31959534 67.62813568 67.93331909 68.10918427
 68.29754639 68.51211548 68.66264343 68.84644318 69.04935455 69.24158478
 69.43386078 69.61502075 69.79949951 69.98509979 70.17510223 70.35868835
 70.5412674  70.72106934 70.89739227 71.07195282 71.24459839 71.41500092
 71.58418274 71.7518158  71.91763306 72.08187866 72.24446869 72.40560913
 72.56526947]
[180.32       162.77388    166.44166565 161.59089661 159.17247009
 170.48130798 168.54507446 162.95230103 166.60253906 160.81478882
 171.15745544 187.80686951 177.99861145 161.3505249  162.35533142
 164.0070800

In [29]:
#get res for all companies; and append in a list
companylist=[]
finalval=[]
for companies in un:
    for i in range(31):
        companylist.append(companies)


for val in alllist:
    for i in val:
        #print(i)
        finalval.append(i)
#finalval        
print(len(finalval))
print(len(companylist))

1147
1147


In [31]:
newdf=pd.DataFrame()
newdf['Company']=companylist
newdf['Predictions']=finalval
companychangedict_onemonth={}
for companies in un:
    df=dffull[dffull.company==companies]
    dflastprice_close=df.iloc[-1,3]
    dfpred=newdf[newdf.Company==companies]
    lastpredicted=dfpred.iloc[-1,1]
    change=dflastprice_close-lastpredicted
    companychangedict_onemonth[companies]=change
companychangedict_onemonth
#get companies in descrnding sorted manner to identify the most profitable company in next 30 days projection
import operator
sorted_d = dict(sorted(companychangedict_onemonth.items(), key=operator.itemgetter(1),reverse=True))
print('Dictionary in descending order by value : ',sorted_d)
changedf=pd.DataFrame.from_dict(sorted_d, orient='index')
changedf.columns=['Change from last real stockval t-30 days']
changedf.head()

Dictionary in descending order by value :  {'GOOGL': 248.13443359374992, 'AZO': 200.9807177734374, 'ADBE': 95.86461547851565, 'NVDA': 92.02878295898438, 'NOW': 86.8030419921875, 'VEEV': 58.7242462158203, 'MSFT': 37.91336853027343, 'W': 26.57217712402344, 'DIS': 25.133987426757812, 'ALXN': 21.518851928710944, 'ILTB': 13.675835571289056, 'VIAC': 13.322879219055174, 'AAXN': 11.219202728271483, 'NEE': 11.16069274902344, 'TER': 8.075097656250001, 'CVS': 7.050790405273439, 'ZEN': 6.004155578613279, 'T': 4.838816070556639, 'VTI': 4.8306713867187625, 'VZ': 4.4749678039550815, 'ALB': 3.328550415039061, 'TERP': 3.323144493103026, 'IXUS': -3.0279740905761727, 'CTRE': -3.4219264221191406, 'FDX': -3.5689239501953125, 'BEP': -3.6082814025878918, 'NVO': -6.4852694702148455, 'IRBT': -6.51254577636719, 'URI': -8.605362548828126, 'BTI': -10.201215820312498, 'STZ': -11.923471679687509, 'ALL': -14.801805419921877, 'ETN': -15.190426940917973, 'VRTX': -21.904827880859386, 'ENPH': -38.479744567871094, 'NFLX'

,Change from last real stockval t-30 days
GOOGL,248.134434
AZO,200.980718
ADBE,95.864615
NVDA,92.028783
NOW,86.803042


In [33]:
changedf

,Change from last real stockval t-30 days
GOOGL,248.134434
AZO,200.980718
ADBE,95.864615
NVDA,92.028783
NOW,86.803042
VEEV,58.724246
MSFT,37.913369
W,26.572177
DIS,25.133987
ALXN,21.518852


In [32]:
changedf.to_csv('t-30change-companies-new.csv')

In [ ]:
#there are multiple other external factors which affect the stock prices and this cannot be the best estimator,
#but given past 15 days of trend we tried to find the best possibile stock rates that would be projected,
#if external factors were not in play
